In [7]:
from test_eval import *
from tqdm import tqdm

In [8]:
def load_idxs(path='ids.txt'):
    tests = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip().split("\t")
            tests.append(line)
    return tests

def gen_random_idxs(doc_ids, times=1000, dump_to='ids.txt'):
    import random
    with open(dump_to, "w") as f:
        while times > 0:
            test_docs = random.choices(doc_ids, k=len(doc_ids))
            output = "\t".join([str(i) for i in test_docs]) + "\n"
            f.write(output)
            times -= 1

In [9]:
result_root = "../../exp/result_bert_base_uncased_5_15/15"
training_sets = ["billsum", "scientific_papers", "cnn_dailymail", "big_patent"]
method = "pref_ordered"
concensus_based_on="mean"
level = "summary"
metrics_newsroom = "Relevance" # "Fluency" # "Coherence" #, "Informativeness", "Fluency", "Relevance"
correlation_type = "spearmanr"

scores = load_newsroom_and_ours("./newsroom-human-eval.csv", "./predictions/metric_bleu.tsv")
if not os.path.exists("ids.txt"):
    gen_random_idxs(list(scores.keys()))

tests = load_idxs()

In [10]:
targetA = [(train, method) for train in training_sets]
targetB = ['metric_blanc_tune', 'metric_supert', 'metric_summaqa_avg_fscore', 'metric_summaqa_avg_prob']

In [11]:
def sig_test():
    f = open("sig_{}.tsv".format(metrics_newsroom), "w", encoding="utf-8")
    f.write(metrics_newsroom + "\n")
    for sysA in targetA:
        scoresA = load_newsroom_and_ours("./newsroom-human-eval.csv", os.path.join(result_root, sysA[0], sysA[1], "test_results_newsroom.tsv"))
        f.write(sysA[0] + "\t")
        for sysB in targetB:
            print(sysA, sysB)
            scoresB = load_newsroom_and_ours("./newsroom-human-eval.csv", os.path.join("predictions", sysB+".tsv"))
            corrA = summary_judge(scoresA, [metrics_newsroom], ["Ours"], concensus_based_on, [correlation_type])
            corrB = summary_judge(scoresB, [metrics_newsroom], ["Ours"], concensus_based_on, [correlation_type])
            if corrA[correlation_type][(metrics_newsroom, "Ours")] < corrB[correlation_type][(metrics_newsroom, "Ours")]: 
                print("Skip")
                f.write("-\t")
                continue
            
            count = 0
            for test in tqdm(tests):
                corrA = summary_judge(scoresA, [metrics_newsroom], ["Ours"], concensus_based_on, [correlation_type], test)
                corrB = summary_judge(scoresB, [metrics_newsroom], ["Ours"], concensus_based_on, [correlation_type], test)
                if corrA[correlation_type][(metrics_newsroom, "Ours")] < corrB[correlation_type][(metrics_newsroom, "Ours")]: count += 1
            pvalue = count / len(tests)
            print("P-value:", pvalue)
            f.write("%.2f\t" % pvalue)
        f.write("\n")

    f.write("\t".join([""] + targetB))
    f.close()

In [12]:
sig_test()

('billsum', 'pref_ordered') metric_blanc_tune


100%|██████████| 1000/1000 [01:56<00:00,  8.55it/s]


P-value: 0.101
('billsum', 'pref_ordered') metric_supert


100%|██████████| 1000/1000 [01:57<00:00,  8.50it/s]


P-value: 0.098
('billsum', 'pref_ordered') metric_summaqa_avg_fscore


...

P-value: 0.189
('big_patent', 'pref_ordered') metric_summaqa_avg_fscore


100%|██████████| 1000/1000 [01:57<00:00,  8.48it/s]


P-value: 0.001
('big_patent', 'pref_ordered') metric_summaqa_avg_prob


100%|██████████| 1000/1000 [01:57<00:00,  8.49it/s]

P-value: 0.0
